<a href="https://colab.research.google.com/github/RYU-MCFLY/-analyst_survival_public/blob/main/Working_Analyst_Survival_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Puede necesitar instalar alguna de estas librerías, solo descomentenlas a necesidad

!pip install lifelines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 349 kB 7.5 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4048 sha256=047c544ab3c565ab26de609d7164d44fbb8e9439cecd23fe65983b73ed124963
  Stored in directory: /root/.cache/pip/wheels/9f/01/ee/1331593abb5725ff7d8c1333aee93a50a1c29d6ddda9665c9f
Successfully built autograd-gamma
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.1.5 requires typing-extensions<4.2.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 

### Objetivo

Implementar el análisis de supervivencia a datos sobre prestamos, para esto veremos:
  1. Estimador Kaplan Meier
  2. Modelo de Cox
  3. Mención de Survival Random Forest

### Carga

In [2]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns 

from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.statistics import logrank_test

In [3]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


Los datos de los prestamos pueden ser encontrados [aquí](https://www.kaggle.com/datasets/yousuf28/prosper-loan). La parte de las etiquetas fue obtenida [aquí](https://github.com/ketra21/prosperloan/blob/master/Prosper%20Loan%20Data%20-%20Variable%20Definitions.csv).

In [4]:
df = pd.read_csv('/content/drive/MyDrive/SCNN-curso/Working_Analyst_Survival_Analysis/prosperLoanData.csv')
df_dict = pd.read_csv('/content/drive/MyDrive/SCNN-curso/Working_Analyst_Survival_Analysis/Prosper Loan Data - Variable Definitions.csv')

FileNotFoundError: ignored

In [ ]:
df_dict

In [ ]:
df_dict_reindex = df_dict.set_index('Variable')

In [ ]:
df_dict_reindex

In [ ]:
df.head()

In [ ]:
df.info()

#### Filtrando columnas

In [ ]:
df_dict_reindex.loc[['LoanKey', 'ProsperRating (numeric)', 'EmploymentStatusDuration',
                    'LoanOriginationDate', 'LoanStatus', 'EmploymentStatus',
                    'ClosedDate', 'Occupation', 'BorrowerState', 'IsBorrowerHomeowner',
                    'StatedMonthlyIncome', 'IncomeRange', 'LoanOriginalAmount'],:]

In [ ]:
df_sub = df.loc[:, ['LoanKey', 'ProsperRating (numeric)', 'EmploymentStatusDuration',
                    'LoanOriginationDate', 'LoanStatus', 'EmploymentStatus',
                    'ClosedDate', 'Occupation', 'BorrowerState', 'IsBorrowerHomeowner',
                    'StatedMonthlyIncome', 'IncomeRange', 'LoanOriginalAmount']]
df_sub.head()

In [ ]:
df_sub.info()

In [ ]:
df_sub['LoanOriginationDate'] = pd.to_datetime(df['LoanOriginationDate'])
df_sub['ClosedDate'] = pd.to_datetime(df_sub['ClosedDate'], errors='ignore')

In [ ]:
df_sub

In [ ]:
df_sub['LoanKey'].unique().shape

In [ ]:
df_sub = df_sub.drop_duplicates(subset='LoanKey').reset_index(drop=True)
df_sub.shape

Revisando el rango máximo de tiempo e ingresos de usuarios

In [ ]:
df_sub[['LoanOriginationDate', 'ClosedDate', 'StatedMonthlyIncome']].max()

In [ ]:
df_sub[['LoanOriginationDate', 'ClosedDate', 'StatedMonthlyIncome']].min()

#### Revisando la variable del evento -> LoanStatus

In [ ]:
df_sub.LoanStatus.unique()

In [ ]:
df_status = df_sub[df_sub['LoanStatus'].isin([ 'Current', 'Chargedoff', 'Defaulted'])].copy()
df_status.head()

In [ ]:
df_status.LoanStatus.unique()

In [ ]:
df_status['LoanStatus_Censored'] = df_status['LoanStatus'].apply(lambda x: 0 if x == 'Current' else 1)

In [ ]:
df_status.head(10)

In [ ]:
max_date = df_status.ClosedDate.max()
max_date

Vamos a arreglar los tiempos para los datos censurados

In [ ]:
df_status.ClosedDate.fillna(max_date, inplace=True)

In [ ]:
df_status

In [ ]:
df_status['time'] = df_status['ClosedDate'] - df_status['LoanOriginationDate']

In [ ]:
df_status.head()

Veamos elementos que tiene problemas en la columna "time" (prestamos creados tarde)

In [ ]:
pd.to_timedelta(0, unit='D')

In [ ]:
df_status[df_status['time'] < pd.to_timedelta(0, unit='D')]

In [ ]:
df_status_clean = df_status[df_status['time'] > pd.to_timedelta(0, unit='D')].copy()
df_status_clean.head()

In [ ]:
df_status_clean['time'] = df_status_clean.time.dt.days

In [ ]:
df_status_clean

In [ ]:
df_status_clean.LoanStatus_Censored.value_counts(normalize=True)

Recordemos: 0 son datos censurados

### Kaplan-Meier

In [ ]:
df_status_clean[['time', 'LoanStatus_Censored']]

In [ ]:
T = df_status_clean.time
E = df_status_clean.LoanStatus_Censored

model = KaplanMeierFitter()

model.fit(T, event_observed=E)

function = model.survival_function_

In [ ]:
function.plot(title='Kaplan Meier Estimator')

In [ ]:
model.median_survival_time_

In [ ]:
model.event_table

In [ ]:
print(model.predict(1000))
print(model.predict(775))

In [ ]:
borrower = (df_status_clean['IsBorrowerHomeowner'] == True)

T = df_status_clean.time
E = df_status_clean.LoanStatus_Censored

model = KaplanMeierFitter()

model.fit(T[borrower], event_observed=E[borrower], label='Homeowner')
fig_1 = model.survival_function_

model.fit(T[~borrower], event_observed=E[~borrower], label='Not Homeowner')
fig_2 = model.survival_function_


figure = pd.concat([fig_1, fig_2], axis=1)
figure.plot(backend='plotly')

In [ ]:
results = logrank_test(T[borrower], T[~borrower], event_observed_A=E[borrower],
                       event_observed_B=E[~borrower])
results.print_summary()

Entonces se puede rechaza la hipótesis nula de que las funciones de riesgo son iguales (por lo tanto tampoco sus funciones de *supervivencia*) 

Es importante señalar que siempre se supone falla!!!

In [ ]:
def survival(data, group_field, time_field, event_field):
  """
  Funcion que recibe un DataFrame y filtra por columna

  Returns
    gráfica de la función de supervivencia (Survival function)
    Logrank Test
  """

  model = KaplanMeierFitter()
  results = []
  con_exp = []

  for i in data[group_field].dropna().unique():
    group = data[data[group_field] == i]
    T = group[time_field]
    E = group[event_field]
    con_exp.append([T, E])
    model.fit(T, E, label=str(i))
    results.append(model.survival_function_)

  survival = pd.concat(results, axis=1)
  
  
  return survival


In [ ]:
rates = survival(df_status_clean, 'Occupation', 'time', 'LoanStatus_Censored')
rates.plot(kind='scatter', title='Pago completo dependiendo de su ocupación', 
           backend='plotly')

In [ ]:
ocupation1 = (df_status_clean['Occupation'] == 'Computer Programmer')
ocupation2 = (df_status_clean['Occupation'] == 'Professor')

T = df_status_clean.time
E = df_status_clean.LoanStatus_Censored

model = KaplanMeierFitter()

model.fit(T[ocupation1], event_observed=E[ocupation1], label='Computer Programmer')
fig_1 = model.survival_function_

model.fit(T[ocupation2], event_observed=E[ocupation2], label='Professor')
fig_2 = model.survival_function_


figure = pd.concat([fig_1, fig_2], axis=1)
figure.plot(backend='plotly',kind='scatter')

In [ ]:
results = logrank_test(T[ocupation1], T[ocupation2], event_observed_A=E[ocupation1],
                       event_observed_B=E[ocupation2])
results.print_summary()

### Cox Model (Segunda Sesión)

In [ ]:
df_status_clean.head()

In [ ]:
to_model = df_status_clean[['LoanOriginalAmount', 'IsBorrowerHomeowner', 'StatedMonthlyIncome', 
                    'time', 'LoanStatus_Censored']]

In [ ]:
model = CoxPHFitter()
model.fit(to_model, duration_col='time', event_col='LoanStatus_Censored')

In [ ]:
model.print_summary()

In [ ]:
model.plot()

In [ ]:
model.plot_partial_effects_on_outcome(covariates='LoanOriginalAmount',
                                      values=[1000, 10000, 
                                              20000], cmap='coolwarm',
                                      figsize=(15, 10))

baseline es $h_0(t)$

Prediciendo riesgos

In [ ]:
to_model[:5]

In [ ]:
model.predict_partial_hazard(to_model[:5])

### DF filter 2009


In [ ]:
df_2009 = df_status_clean[df_status_clean['LoanOriginationDate'].dt.year == 2009].reset_index(drop=True)
df_2009.head()

In [ ]:
df_2009.info()

In [ ]:
# Valores distintos en la columna
df_2009.EmploymentStatus.unique()

In [ ]:
df_2009[['EmploymentStatus']]

In [ ]:
dummies = pd.get_dummies(df_2009[['EmploymentStatus']], drop_first=True)
dummies

In [ ]:
dummies.sum()

In [ ]:
df_2009.IncomeRange.unique()

In [ ]:
def income_range_ordinal(label: str):
  if label in ('$0', 'Not employed'):
    return 0
  elif label == '$1-24,999':
    return 1
  elif label == '$75,000-99,999':
    return 2
  elif label == '$25,000-49,999':
    return 3
  elif label == '$50,000-74,999':
    return 4
  else:
    return 5

In [ ]:
df_2009['IncomeRange_ord'] = df_2009.IncomeRange.apply(income_range_ordinal)

In [ ]:
df_2009.head()

In [ ]:
df_2009_dumm = pd.concat([df_2009, dummies], axis=1)
df_2009_dumm.head()

In [ ]:
df_2009_num = df_2009_dumm._get_numeric_data()
df_2009_num

In [ ]:
null_col = df_2009_num.isnull().sum()
null_col[null_col > 0]

In [ ]:
df_2009_num.dropna(inplace=True)

In [ ]:
df_2009_num.info()

In [ ]:
df_2009_num.columns.duplicated()

In [ ]:
df_2009_num

### Cox Model 2009

In [ ]:
to_model = df_2009_num #.iloc[:, 0:10]

In [ ]:
to_model

In [ ]:
model = CoxPHFitter()
model.fit(to_model, duration_col='time', event_col='LoanStatus_Censored')

model.print_summary()

In [ ]:
model.plot()

In [ ]:
model.baseline_cumulative_hazard_.plot()

In [ ]:
model.baseline_survival_.plot()

In [ ]:
to_model[100:105]

In [ ]:
model.predict_partial_hazard(to_model[100:105])

## Caso con estimador paramétrico

In [ ]:
model = CoxPHFitter(baseline_estimation_method='spline', n_baseline_knots=6)
model.fit(to_model, duration_col='time', event_col='LoanStatus_Censored')

# model.print_summary()
model.baseline_cumulative_hazard_.plot()

In [ ]:
model.baseline_survival_.plot()